# PGLYRP2 - Single Gene Analysis in AMP-PD WGS data


*PGLYRP2 from NCBI gene: hg38 chr19:15468645-15479501*

- Project: Sex Differences in PGLYRP2 Variant rs892145 in Parkinson's Disease
- Version: Python/3.10.12
- Last Updated: 12-JUNE-2025
- Update Description: Updated the gene coordinates (previously Ensemble which turned out to contain an upstream region) and added PC1-5 as covariates in the interaction analysis


## Notebook overview
In this notebook we performed regression analyses with PGLYRP2 gene variants and PD in the WGS AMP-PD data using PLINK

## Description

__1. Description__

-  Getting Started
- Loading Python libraries
- Defining functions
- Set Paths
- Make working directory

__2. Installing packages__

__3. Copy over data__

__4. Create a covariate file with AMP-PD data__

__5. Remove related individuals and PCA__

__6. Annotation of the gene__

__7. Case/Control Analysis__

## Loading Python libraries

In [ ]:
# Use the os package to interact with the environment
import os
import sys

# Bring in Pandas for Dataframe functionality
import pandas as pd
from functools import reduce

# Bring some visualization functionality 
import seaborn as sns  

# numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

## Defining functions

In [ ]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

## Set paths

In [ ]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']
WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name @ `WORKSPACE_NAME`: {WORKSPACE_NAME}')
print(f'Billing Project @ `BILLING_PROJECT_ID`: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data @ `WORKSPACE_BUCKET`: {WORKSPACE_BUCKET}')
print('')

## AMP-PD v3.0
## Explicitly define release v3.0 path as:
# AMP_RELEASE_PATH 
# Also set paths to:
# AMP_CLINICAL_RELEASE_PATH
# AMP_RELEASE_GATK_PATH
# AMP_WGS_RELEASE_PATH
# AMP_WGS_RELEASE_PLINK_PATH
# AMP_WGS_RELEASE_PLINK_PFILES

print('AMP-PD v3.0')
print(f'Path to AMP-PD v3.0 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PFILES}')
print('')

## Make working directory

In [ ]:
# Make a directory called "/home/jupyter/PGLYRP2_AMPPD/"
WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

shell_do(f'mkdir -p {WORK_DIR}') # f' means f-string - contains expressions to execute the code

In [ ]:
%%bash
cd /home/jupyter/PGLYRP2_AMPPD/
ls

## Install Packages

In [ ]:
%%capture
%%bash

# Install plink 1.9
cd /home/jupyter/
if test -e /home/jupyter/plink; then
    echo "Plink is already installed in /home/jupyter/"
else
    echo "Plink is not installed"
    cd /home/jupyter

    wget http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 

    unzip -o plink_linux_x86_64_20190304.zip
    mv plink plink1.9
fi

In [ ]:
%%bash

# chmod plink 1.9 to make sure you have permission to run the program
chmod u+x /home/jupyter/plink1.9

In [ ]:
%%capture
%%bash

# Install plink 2.0
cd /home/jupyter/
if test -e /home/jupyter/plink2; then

echo "Plink2 is already installed in /home/jupyter/"
else
echo "Plink2 is not installed"
cd /home/jupyter/

wget http://s3.amazonaws.com/plink2-assets/plink2_linux_x86_64_latest.zip

unzip -o plink2_linux_x86_64_latest.zip

fi

In [ ]:
%%bash

# chmod plink 2 to make sure you have permission to run the program
chmod u+x /home/jupyter/plink2

In [ ]:
%%capture
%%bash

# Install ANNOVAR: We are adding the download link after registration on the annovar website
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/annovar; then

echo "annovar is already installed in /home/jupyter/notebooks"
else
echo "annovar is not installed"
cd /home/jupyter/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

In [ ]:
%%capture
%%bash

# Install BCFtools

if test -e /home/jupyter/bcftools; then
    echo "BCFtools is already installed in /home/jupyter/bcftools"
else
    echo "BCFtools is not installed"
    cd /home/jupyter/

    # Download the latest version of BCFtools
    wget https://github.com/samtools/bcftools/releases/download/1.21/bcftools-1.21.tar.bz2

    # Extract the downloaded file
    tar -xvjf bcftools-1.21.tar.bz2

    # Move into the extracted directory
    cd bcftools-1.21

    # Compile and install BCFtools
    make
    make install

    # Move the installed BCFtools to a specific directory
    mv bcftools /home/jupyter/bcftools
fi

In [ ]:
%%bash

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20170905 humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp47a humandb/ 


In [ ]:
%%bash

# Install RVTESTS: Option 1 (~15min)
if [ ! -e /home/jupyter/tooles/rvtests ]; then
    echo "RVTESTS not found. Installing..."
    mkdir -p /home/jupyter/tools/rvtests
    cd /home/jupyter/tools/rvtests

    wget https://github.com/zhanxw/rvtests/releases/download/v2.1.0/rvtests_linux64.tar.gz 

    tar -zxvf rvtests_linux64.tar.gz
else
    echo "RVTESTS is already installed."
fi

In [ ]:
%%bash
cd /home/jupyter/tools/rvtests/executable
ls

In [ ]:
! chmod 777 /home/jupyter/tools/rvtests/executable/rvtest

In [ ]:
! chmod 777 /home/jupyter/tools/rvtests/executable/rvtest

In [ ]:
%%bash

/home/jupyter/tools/rvtests/executable/rvtest --help

In [ ]:
%%bash
cd /home/jupyter/tools/
ls

## Copy Over Files

In [ ]:
# Check directory where AMP-PD data is
print("List available imputed genotype information in AMPPD")
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {AMP_WGS_RELEASE_PLINK_PFILES}')

In [ ]:
##  AMP-PD release 3 data
# PGLYRP2 from NCBI gene: hg38 (chr19:15468645-15498956)

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {AMP_WGS_RELEASE_PLINK_PFILES}/chr19.* {WORK_DIR}')


In [ ]:
#Copy other AMP-PD files (clinical info)

shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {AMP_RELEASE_PATH}/amp_pd_case_control.csv {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {AMP_CLINICAL_RELEASE_PATH}/Enrollment.csv {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {AMP_CLINICAL_RELEASE_PATH}/Demographics.csv {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {AMP_CLINICAL_RELEASE_PATH}/PD_Medical_History.csv  {WORK_DIR}')

In [ ]:
##refFlat

shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {WORKSPACE_BUCKET}/refFlat.txt {WORK_DIR}')

In [ ]:
## hg38.fa

shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {WORKSPACE_BUCKET}/hg38.fa {WORK_DIR}')

In [ ]:
%%bash

WORK_DIR='/home/jupyter/PGLYRP2_AMPPD/'
cd $WORK_DIR

wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
gunzip -k hg38.fa.gz

In [ ]:
%%bash
## Check files
cd /home/jupyter/PGLYRP2_AMPPD/
ls

# Create a covariate file with AMP-PD data


In [ ]:
# load clinical information
pd_case_control_df = pd.read_csv(f'{WORK_DIR}/amp_pd_case_control.csv')

# visualize
#pd_case_control_df

In [ ]:
# Keep columns of interest
pd_case_control_latest_df = pd_case_control_df[['participant_id', 'diagnosis_latest', 'case_control_other_latest']].copy()

# Rename Columns
pd_case_control_latest_df.columns = ['ID', 'LATEST_DX', 'CASE_CONTROL']


In [ ]:
#Check case/control value counts
print(pd_case_control_latest_df['CASE_CONTROL'].value_counts())

In [ ]:
# Add column for study origin
pd_case_control_latest_df['COHORT']= np.where(pd_case_control_latest_df.ID.str.contains("LB-"), "LBD",
                                    np.where(pd_case_control_latest_df.ID.str.contains("PP-"), "PPMI",
                                    np.where(pd_case_control_latest_df.ID.str.contains("PD-"), "PDBP",
                                    np.where(pd_case_control_latest_df.ID.str.contains("HB-"), "HBS",
                                    np.where(pd_case_control_latest_df.ID.str.contains("LC-"), "LCC",
                                    np.where(pd_case_control_latest_df.ID.str.contains("BF-"), "BIOFIND",
                                    np.where(pd_case_control_latest_df.ID.str.contains("SU-"), "SURE-PD3",
                                    np.where(pd_case_control_latest_df.ID.str.contains("SY-"), "STEADY-PD3", np.nan))))))))


In [ ]:
# Drop duplicates
case_con_reduced = pd_case_control_latest_df.copy()
case_con_reduced.drop_duplicates(subset=['ID'], inplace=True)

In [ ]:
#Assign case control for Plink
conditions = [
    (case_con_reduced['CASE_CONTROL'] == "Case"),
    (case_con_reduced['CASE_CONTROL'] == "Control")]
## Assign cases=2, controls=1, -9 Other (as used in PLINK)
choices = [2,1]
case_con_reduced['PHENO'] = np.select(conditions, choices, default=-9).astype(np.int64)

# set indices
case_con_reduced.reset_index(inplace=True)
case_con_reduced.drop(columns=["index"], inplace=True)

# Remove initial Pheno Column
case_con_reduced.drop(columns=['CASE_CONTROL'], inplace=True)


In [ ]:
# Phenotype counts
print(case_con_reduced['PHENO'].value_counts(dropna=False))

In [ ]:
# Load Enrollment.csv
enrollment_df = pd.read_csv(f'{WORK_DIR}/Enrollment.csv')

In [ ]:
# Keep columns of interest
enrollment_subset_df = enrollment_df[['participant_id', 'study_arm']].copy()

# Rename columns
enrollment_subset_df.columns = ['ID', 'ENROLL_STUDY_ARM']
enrollment_subset_df.head()

# Drop duplicates
enrollment_subset_df.drop_duplicates(subset=['ID'], keep='first', inplace=True)

In [ ]:
# load demographic data
demographics_df = pd.read_csv(f'{WORK_DIR}/Demographics.csv')

In [ ]:
# Rename Columns
demographics_df.rename(columns = {'participant_id':'ID'}, inplace = True)
demographics_df.rename(columns = {'age_at_baseline':'BASELINE_AGE'}, inplace = True)
demographics_df.rename(columns = {'race':'RACE'}, inplace = True)
demographics_df.rename(columns = {'ethnicity':'ETHNICITY'}, inplace = True)

In [ ]:
# Sort by visit month and Drop Duplicates
demographics_baseline_df = demographics_df \
.sort_values('visit_month', ascending=True) \
.drop_duplicates('ID').sort_index()

In [ ]:
# Merge last diagnostic with diagnostic at enrollement
demographics_df_casecon = demographics_df.merge(case_con_reduced, on='ID', how='outer')

In [ ]:
#Recode Sex Columns
conditions = [
     (demographics_df_casecon['sex'] == "Male"),
     (demographics_df_casecon['sex'] == "Female")]

# 1=Male; 2=Female
choices = [1,2]

# Create new column Sex
demographics_df_casecon['SEX'] = np.select(conditions, choices, default=None).astype(np.int64)

# Remove previous Sex column
demographics_df_casecon.drop(columns=['sex'], inplace=True)

In [ ]:
## Keep only columns of interest
demographics_df_casecon_toKeep = demographics_df_casecon[['ID', 'PHENO', 'SEX', 'RACE',
                                                          'ETHNICITY','BASELINE_AGE', 'LATEST_DX',
                                                          'COHORT']].copy()

In [ ]:
# Merge Pheno with demograhic data
enrollment_pheno_df = demographics_df_casecon_toKeep.merge(enrollment_subset_df, on='ID', how='outer')

# Create FID column
enrollment_pheno_df['FID'] = enrollment_pheno_df['ID'].values

# rename ID as IID 
enrollment_pheno_df.rename(columns = {'ID':'IID'}, inplace = True)

# Order columns
reorder_enrollment_pheno_df = enrollment_pheno_df[['FID', 'IID', 'PHENO',
                                                  'SEX', 'RACE','ETHNICITY', 'BASELINE_AGE', 'LATEST_DX',
                                                  'COHORT', 'ENROLL_STUDY_ARM']].copy()

In [ ]:
#Remove individuals from the genetic registry cohorts and other enrollment categories
#We only want to keep  controls who were originally enrolled as controls, and PD cases originally enrolled as cases
#Will exclude prodromal, SWEDD and unknown individuals also
#Individuals enrolled as disease control but the latest diagnosis is PD will also be excluded

#Keep only individuals enrolled as Healthy Control/Disease Control or PD.
filtered_enrollment_pheno_df = reorder_enrollment_pheno_df.copy()
filtered_enrollment_pheno_df = filtered_enrollment_pheno_df[filtered_enrollment_pheno_df['ENROLL_STUDY_ARM'].isin(['Disease Control', 
                                                                                                                   'Healthy Control', 
                                                                                                                  'PD'])]
#Now remove individuals with PHENO of -9 (keep only individuals with PHENO of 1 or 2)
filtered_enrollment_pheno_df = filtered_enrollment_pheno_df[filtered_enrollment_pheno_df['PHENO'].isin([1,2])]

#Now remove individuals who were enrolled with the opposite diagnosis, i.e. individuals who were enrolled as controls but have a latest diagnosis of PD
filtered_enrollment_pheno_df = filtered_enrollment_pheno_df[((filtered_enrollment_pheno_df['PHENO'] == 2) & (filtered_enrollment_pheno_df['ENROLL_STUDY_ARM'] == 'PD')) | (filtered_enrollment_pheno_df['PHENO'] == 1)]


#Check value counts again
filtered_enrollment_pheno_df.groupby(['PHENO', 'ENROLL_STUDY_ARM']).size().reset_index(name='counts')


In [ ]:
#Save file - this is intermediate not final covariate file
#This includes all ancestries
filtered_enrollment_pheno_df.to_csv(f'{WORK_DIR}/COVS_temp.txt', index=False, sep='\t', na_rep='NA')

## Import GenoTools ancestry

This is needed for the ancestry labels. Need to do this before running the PCA as the PCs should be generated just in the population/group of interest.

In [ ]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {WORKSPACE_BUCKET}/AMPPD_v3_COV_wPHENOS_wGENOTOOLS.csv {WORK_DIR}')


In [ ]:
#Read covariate file to get ancestry groups
ancestries = pd.read_csv(f'{WORK_DIR}/AMPPD_v3_COV_wPHENOS_wGENOTOOLS.csv')

In [ ]:
#This file has a lot of the diagnosis info that we have generated ourselves earlier.
#We just want the ancestry label (the last column)

ancestries_selected = ancestries[['ID', 'label']]

In [ ]:
#Filter for ancestry of interest
#Change this to your ancestry of interest
ancestry_toKeep = 'EUR'

ancestries_keep = ancestries_selected.copy()
ancestries_keep = ancestries_selected[ancestries_selected['label'] == ancestry_toKeep]

In [ ]:
#Check the value counts again - this should just be the ancestry you are interested in
ancestries_keep.label.value_counts()

In [ ]:
#Merge covariate file with the ancestry file
#Inner join - meaning we only want to keep individuals in both dataframes
#So this will keep just individuals in the selected ancestry group
merged = pd.merge(filtered_enrollment_pheno_df, ancestries_keep, left_on = 'IID', right_on = 'ID', how = 'inner')


In [ ]:
#Write list of individuals from the ancestry group of interest to keep from the genetic dataset
#We need to do this before generating PCs
individuals_toKeep = merged[['FID', 'IID']]


In [ ]:
#Export file

#Note this still has related individuals
#We will remove related individuals at the next step

individuals_toKeep.to_csv('/home/jupyter/PGLYRP2_AMPPD/individuals_toKeep.EUR_enroll.txt', index=False, sep='\t')

# Remove related individuals

In [ ]:
#From the workspace bucket, copy over the file with EUR related individuals to remove (just one individual from each pair)
#Make sure to copy over the file for the correct ancestry
WORK_DIR='/home/jupyter/PGLYRP2_AMPPD/'
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {WORKSPACE_BUCKET}/toRemove_1stand2ndDegree_Relateds_EUR.txt {WORK_DIR}')


In [ ]:
# Remove related individuals and keep only PD and controls in the correct enrollment categories
# Used make-bed not make-pgen because there is a bug in plink2 and it doesn't write the pgen file https://groups.google.com/g/plink2-users/c/z_ZVhacpnts/m/byzJxGl5AwAJ
# Have included max-alleles 2 because plink cannot write bim file with multialleleic variants
WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'
! /home/jupyter/tools/plink2 \
--pfile {WORK_DIR}/chr19 \
--keep {WORK_DIR}/individuals_toKeep.EUR_enroll.txt \
--remove {WORK_DIR}/toRemove_1stand2ndDegree_Relateds_EUR.txt \
--make-bed \
--max-alleles 2 \
--out {WORK_DIR}/chr19_nonrelated_enroll

In [ ]:
# Download previously created covar file from workspace bucket 
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORKSPACE_BUCKET}/AMPPD_EUR.COVS.txt {WORK_DIR}')

### Check mean age and SD for cases and controls 

In [ ]:
#Check mean age and SD for cases and controls 

ancestries = {'AMPPD'}

for ancestry in ancestries:
    final_df = pd.read_csv(f'/home/jupyter/PGLYRP2_AMPPD/AMPPD_EUR.COVS.txt', sep='\t')
    print(f'WORKING ON: {ancestry}')

    # Check value counts of pheno
    pheno = final_df['PHENO'].value_counts(dropna=False)
    print(f'Pheno: {pheno}')
    
    mean_age = final_df.groupby('PHENO')['AGE'].agg(['mean', 'std'])
    print(f'Mean age and SD: {mean_age}')

    sex_pct = pd.crosstab(final_df['PHENO'], final_df['SEX'], normalize='index') * 100
    print(f'Sex %: {sex_pct}')
    

## Update sex and pheno info in plink genetic files


In [ ]:
%%bash
WORK_DIR='/home/jupyter/PGLYRP2_AMPPD/'
cd $WORK_DIR
head chr19_nonrelated_enroll.fam

In [ ]:
# Update sex and phenotype in plink files, using the AMPPD_EUR.COVS.txt file that we made
#-update-sex and column number tells plink where to look for the sex information in the file

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/chr19_nonrelated_enroll \
--update-sex {WORK_DIR}/AMPPD_EUR.COVS.txt col-num=5 \
--pheno {WORK_DIR}/AMPPD_EUR.COVS.txt \
--pheno-name PHENO \
--make-pgen \
--out {WORK_DIR}/chr19_nonrelated_enroll.updated

In [ ]:
%%bash
WORK_DIR='/home/jupyter/PGLYRP2_AMPPD/'
cd $WORK_DIR
head chr19_nonrelated_enroll.updated.psam

# Extract and annotate the gene

## Extract the region using PLINK

- Extract *PGLYRP2* gene 
- *PGLYRP2* coordinates: chr19:15468645-15479501

In [ ]:
## update the variant IDs
! /home/jupyter/tools/plink2 \
--pfile /home/jupyter/PGLYRP2_AMPPD/chr19_nonrelated_enroll.updated\
--fa /home/jupyter/PGLYRP2_AMPPD/hg38.fa \
--memory 25000 \
--set-all-var-ids "chr@:#:\$r:\$a" \
--new-id-max-allele-len 999 --sort-vars \
--make-pgen \
--out /home/jupyter/PGLYRP2_AMPPD/chr19_nonrelated_enroll.updated_formatted

In [ ]:
## extract region using plink and make pgen files
#PGLYRP2 coordinates: chr19:15468645-15498956
WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

! /home/jupyter/tools/plink2 \
--pfile {WORK_DIR}/chr19_nonrelated_enroll.updated_formatted \
--chr 19 \
--from-bp 15468645 \
--to-bp 15479501 \
--max-alleles 2 \
--mac 2 \
--hwe 0.0001 \
--make-pgen \
--out {WORK_DIR}/PGLYRP2

In [ ]:
## extract region using plink and make bed files
#PGLYRP2 coordinates: chr19:15468645-15498956
WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'
    
! /home/jupyter/plink2 \
--pfile {WORK_DIR}/chr19_nonrelated_enroll.updated_formatted \
--chr 19 \
--from-bp 15468645 \
--to-bp 15479501 \
--mac 2 \
--max-alleles 2 \
--hwe 0.0001 \
--make-bed \
--out {WORK_DIR}/PGLYRP2


In [ ]:
# Visualize bim file
! head /home/jupyter/PGLYRP2_AMPPD/PGLYRP2.bim

In [ ]:
# Visualize fam file
! head /home/jupyter/PGLYRP2_AMPPD/PGLYRP2.fam

## Turn binary files into VCF

In [ ]:
## Turn binary files into VCF for annovar input
WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/PGLYRP2 \
--recode vcf id-paste=iid \
--out {WORK_DIR}/PGLYRP2


In [ ]:
### Bgzip and Tabix (zip and index the file)
for ancestry in ancestries:    
    ! bgzip -f {WORK_DIR}/PGLYRP2.vcf
    ! tabix -f -p vcf {WORK_DIR}/PGLYRP2.vcf.gz 
    

## Annotate using ANNOVAR

In [ ]:
## annotate using ANNOVAR
WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

! perl /home/jupyter/tools/annovar/table_annovar.pl {WORK_DIR}/PGLYRP2.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out {WORK_DIR}/PGLYRP2.annovar \
-remove -protocol refGene,clinvar_20170905,dbnsfp47a \
-operation g,f,f \
--nopolish \
-nastring . \
-vcfinput


In [ ]:
test = pd.read_csv('/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.annovar.hg38_multianno.txt',sep='\t')
test.head()

In [ ]:

    # Read in ANNOVAR multianno file
    gene = pd.read_csv(f'/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.annovar.hg38_multianno.txt', sep = '\t')
    
    #Filter for the correct gene name (sometimes other genes are also included)
    gene = gene[gene['Gene.refGene'] == 'PGLYRP2']
    
    # Convert the CADD scores to float, set errors to NaN
    gene['CADD_phred'] = pd.to_numeric(gene['CADD_phred'], errors='coerce')

    #Print number of variants in the different categories
    results = [] 

    intronic = gene[gene['Func.refGene']== 'intronic']
    upstream = gene[gene['Func.refGene']== 'upstream']
    downstream = gene[gene['Func.refGene']== 'downstream']
    utr5 = gene[gene['Func.refGene']== 'UTR5']
    utr3 = gene[gene['Func.refGene']== 'UTR3']
    splicing = gene[gene['Func.refGene']== 'splicing']
    exonic = gene[gene['Func.refGene']== 'exonic']
    stopgain = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'stopgain')]
    stoploss = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'stoploss')]
    startloss = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'startloss')]
    frameshift_deletion = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'frameshift deletion')]
    frameshift_insertion = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'frameshift insertion')]
    nonframeshift_deletion = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'nonframeshift deletion')]
    nonframeshift_insertion = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'nonframeshift insertion')]
    coding_nonsynonymous = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
    coding_synonymous = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'synonymous SNV')]
        
    print('AMPPD')
    print('Total variants: ', len(gene))
    print("Intronic: ", len(intronic))
    print("Upstream: ", len(upstream))
    print("Downstream: ", len(downstream))
    print('UTR3: ', len(utr3))
    print('UTR5: ', len(utr5))
    print("Splicing: ", len(splicing))
    print("Total exonic: ", len(exonic))
    print("Stopgain: ", len(stopgain))
    print("Stoploss: ", len(stoploss))
    print("Startloss: ", len(startloss))
    print("Frameshift deletion: ", len(frameshift_deletion))
    print("Frameshift insertion: ", len(frameshift_insertion))
    print("Non-frameshift insertion: ", len(nonframeshift_insertion))
    print("Non-frameshift deletion: ", len(nonframeshift_deletion))
    print('Synonymous: ', len(coding_synonymous))
    print("Nonsynonymous: ", len(coding_nonsynonymous))
    results.append((gene, intronic, upstream, downstream, utr3, utr5, splicing,exonic,stopgain,stoploss,startloss, frameshift_deletion,frameshift_insertion,nonframeshift_deletion,nonframeshift_insertion,coding_synonymous, coding_nonsynonymous))
    print('\n')
    
    ## For rvtests
    
    # Potential functional: These are variants annotated as frameshift, nonframeshift, startloss, stoploss, stopgain, splicing, missense, exonic, UTR5, UTR3, upstream (-100bp), downstream (+100bp), or ncRNA. 
    potentially_functional = gene[gene['Func.refGene'] != 'intronic']
    # Coding: These are variants annotated as frameshift, nonframeshift, startloss, stoploss, stopgain, splicing, or missense.
    coding_variants = gene[(gene['Func.refGene'] == 'splicing') | (gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] != 'synonymous SNV')]
    # Loss of function: These are variants annotated as frameshift, startloss,stopgain, or splicing.
    loss_of_function = gene[(gene['Func.refGene'] == 'splicing') | (gene['ExonicFunc.refGene'] == 'stopgain') | (gene['ExonicFunc.refGene'] == 'startloss') | (gene['ExonicFunc.refGene'] == 'frameshift deletion') | (gene['ExonicFunc.refGene'] == 'frameshift insertion')]
    
    
    # Save in PLINK format
    variants_toKeep = potentially_functional[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
    variants_toKeep.to_csv(f'/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.potentially_functional.variantstoKeep.txt', sep="\t", index=False, header=False)

    variants_toKeep2 = coding_variants[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
    variants_toKeep2.to_csv(f'/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.coding_variants.variantstoKeep.txt', sep="\t", index=False, header=False)

    variants_toKeep3 = loss_of_function[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
    variants_toKeep3.to_csv(f'/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.loss_of_function.variantstoKeep.txt', sep="\t", index=False, header=False)
        
    maf_01 = gene[gene['Otherinfo1'] < 0.01]
    variants_toKeep4 = maf_01[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
    variants_toKeep4.to_csv(f'/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.maf_01.variantstoKeep.txt', sep="\t", index=False, header=False)

    maf_03 = gene[gene['Otherinfo1'] < 0.03]
    variants_toKeep5 = maf_03[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
    variants_toKeep5.to_csv(f'/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.maf_03.variantstoKeep.txt', sep="\t", index=False, header=False)
    

    # For assoc
    
    # These are all exonic variants
    exonic = gene[gene['Func.refGene'] == 'exonic']
    
    # Save in PLINK format
    variants_toKeep7 = exonic[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
    variants_toKeep7.to_csv(f'/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.exonic.variantstoKeep.txt', sep="\t", index=False, header=False)
    

In [ ]:
!head /home/jupyter/PGLYRP2_AMPPD/PGLYRP2.maf_03.variantstoKeep.txt

## Burden Analyses using RVTests

In [ ]:
#Loop over the different variant classes, extract variants and create the right format for RVTest
variant_classes = ['potentially_functional', 'coding_variants','loss_of_function','maf_01','maf_03']

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

#Loop over all the ancestries and the 3 variant classes
for ancestry in ancestries:
    for variant_class in variant_classes:
                
        # Print the command to be executed (for debugging purposes)
        print(f'Running plink to extract {variant_class} variants for ancestry: {ancestry}')
        
        #Extract relevant variants
        ! /home/jupyter/plink2 \
        --pfile {WORK_DIR}/PGLYRP2 \
        --extract range {WORK_DIR}/PGLYRP2.{variant_class}.variantstoKeep.txt \
        --recode vcf-iid \
        --out {WORK_DIR}/PGLYRP2.{variant_class}
        
        # Print the command to be executed (for debugging purposes)
        print(f'Running bgzip and tabix for {variant_class} variants for ancestry AMP-PD')
        
        ## Bgzip and Tabix (zip and index the file)
        ! bgzip -f {WORK_DIR}/PGLYRP2.{variant_class}.vcf
        ! tabix -f -p vcf {WORK_DIR}/PGLYRP2.{variant_class}.vcf.gz
        

In [ ]:
#Loop over the different variant classes 
#Run with all covariates (including age)
#Run RVtests

variant_classes = ['potentially_functional', 'coding_variants','loss_of_function','maf_01','maf_03']

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'


for variant_class in variant_classes:
    
    # Print the command to be executed (for debugging purposes)
    print(f'Running RVtests for {variant_class} variants for ancestry AMP-PD')
        
    ## RVtests with covariates 
    #Make sure the pheno and covariate file starts with the first 5 columsn: fid, iid, fatid, matid, sex
    #The pheno-name flag only works when the pheno/covar file is structured properly
    ! /home/jupyter/tools/rvtests/executable/rvtest --noweb --hide-covar \
    --out {WORK_DIR}/PGLYRP2.burden.{variant_class} \
    --kernel skat,skato \
    --inVcf {WORK_DIR}/PGLYRP2.{variant_class}.vcf.gz \
    --pheno {WORK_DIR}/AMPPD_EUR.COVS.txt \
    --pheno-name PHENO \
    --gene PGLYRP2 \
    --geneFile {WORK_DIR}/refFlat.txt \
    --covar {WORK_DIR}/AMPPD_EUR.COVS.txt \
    --covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5
        

In [ ]:
! cat {WORK_DIR}/PGLYRP2.burden.potentially_functional.Skat.assoc

In [ ]:
! cat {WORK_DIR}/PGLYRP2.burden.coding_variants.Skat.assoc

In [ ]:
! cat {WORK_DIR}/PGLYRP2.burden.maf_03.Skat.assoc

No loss of function

In [ ]:
! cat {WORK_DIR}/PGLYRP2.burden.potentially_functional.SkatO.assoc

In [ ]:
! cat {WORK_DIR}/PGLYRP2.burden.coding_variants.SkatO.assoc

In [ ]:
! cat {WORK_DIR}/PGLYRP2.burden.maf_03.SkatO.assoc

In [ ]:
#Loop over the different variant classes 
#Run without age as a covariate (this has been done for the GP2 data and we want to do the same also here)
#Run RVtests

variant_classes = ['potentially_functional', 'coding_variants','loss_of_function','maf_01','maf_03']

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'


for variant_class in variant_classes:
    
    # Print the command to be executed (for debugging purposes)
    print(f'Running RVtests for {variant_class} variants for ancestry AMP-PD')
        
    ## RVtests with covariates 
    #Make sure the pheno and covariate file starts with the first 5 columsn: fid, iid, fatid, matid, sex
    #The pheno-name flag only works when the pheno/covar file is structured properly
    ! /home/jupyter/tools/rvtests/executable/rvtest --noweb --hide-covar \
    --out {WORK_DIR}/PGLYRP2.burden.{variant_class}_NOAGE \
    --kernel skat,skato \
    --inVcf {WORK_DIR}/PGLYRP2.{variant_class}.vcf.gz \
    --pheno {WORK_DIR}/AMPPD_EUR.COVS.txt \
    --pheno-name PHENO \
    --gene PGLYRP2 \
    --geneFile {WORK_DIR}/refFlat.txt \
    --covar {WORK_DIR}/AMPPD_EUR.COVS.txt \
    --covar-name SEX,PC1,PC2,PC3,PC4,PC5
        

In [ ]:
! cat {WORK_DIR}/PGLYRP2.burden.potentially_functional_NOAGE.Skat.assoc

In [ ]:
! cat {WORK_DIR}/PGLYRP2.burden.coding_variants_NOAGE.Skat.assoc

In [ ]:
! cat {WORK_DIR}/PGLYRP2.burden.maf_03_NOAGE.Skat.assoc

In [ ]:
! cat {WORK_DIR}/PGLYRP2.burden.potentially_functional_NOAGE.SkatO.assoc

In [ ]:
! cat {WORK_DIR}/PGLYRP2.burden.coding_variants_NOAGE.SkatO.assoc

In [ ]:
! cat {WORK_DIR}/PGLYRP2.burden.maf_03_NOAGE.SkatO.assoc

## Case/Control Analysis

### Glossary

- CHR Chromosome code
- SNP Variant identifier
- A1 Allele 1 (usually minor)
- A2 Allele 2 (usually major)
- MAF Allele 1 frequency in all subjects
- F_A/MAF_A Allele 1 frequency in cases
- F_U/MAF_U Allele 1 frequency in controls
- NCHROBS_A Number of case allele observations
- NCHROBS_U Number of control allele observations

## ALL VARIANTS
#### assoc

In [ ]:
#Run case-control analysis using plink assoc for ALL variants, not adjusting for any covariates

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

!/home/jupyter/plink1.9 \
--bfile {WORK_DIR}/PGLYRP2 \
--assoc \
--maf 0.01 \
--mac 2 \
--hwe 0.0001 \
--adjust \
--allow-no-sex \
--ci 0.95 \
--out {WORK_DIR}/PGLYRP2.allvariants
    
#--recode A creates a new text fileset, showing each variant in each case and control for the minor allele (A).
! /home/jupyter/plink1.9 \
--bfile {WORK_DIR}/PGLYRP2 \
--recode A \
--out {WORK_DIR}/PGLYRP2.allvariants

In [ ]:
#Process results from plink assoc unadjusted analysis
#As there are very few or no significant variants with p-value < 0.05 - we will save results dataframe of all coding variants

#Look at assoc results
freq = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants.assoc', delim_whitespace=True)
    
#Filter for significant variants p < 0.05 - if any
sig_all_nonadj = freq[freq['P']<0.05]
    
print(f'There are {len(sig_all_nonadj)} variants with p-value < 0.05')

#Read in plink recoded data (.raw file)
recode = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants.raw', delim_whitespace=True)

# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in AMP-PD for PGLYRP2: {len(variants)}')

# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]

results = []

# Pre-filter the dataset
total_cases = cases_data.shape[0]
total_controls = controls_data.shape[0]
results = []

for variant in variants:
    ## For PD cases
    hom_cases = (cases_data[variant] == 2).sum()
    het_cases = (cases_data[variant] == 1).sum()
    hom_ref_cases = (cases_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_cases = total_cases - (hom_cases + het_cases + hom_ref_cases)  # Missing data count
    freq_cases = (2 * hom_cases + het_cases) / (2 * (total_cases - missing_cases))  # Adjust for missing data in denominator

    ## For controls
    hom_controls = (controls_data[variant] == 2).sum()
    het_controls = (controls_data[variant] == 1).sum()
    hom_ref_controls = (controls_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_controls = total_controls - (hom_controls + het_controls + hom_ref_controls)  # Missing data count
    freq_controls = (2 * hom_controls + het_controls) / (2 * (total_controls - missing_controls))  # Adjust for missing data in denominator
    
    # Append results in dictionary format
    results.append({
        'Variant': variant,
        'Hom Cases': hom_cases,
        'Het Cases': het_cases,
        'Hom Ref Cases': hom_ref_cases,
        'Missing Cases': missing_cases,
        'Total Cases': total_cases,
        'Carrier Freq in Cases': freq_cases,
        'Hom Controls': hom_controls,
        'Het Controls': het_controls,
        'Hom Ref Controls': hom_ref_controls,
        'Missing Controls': missing_controls,
        'Total Controls': total_controls,
        'Carrier Freq in Controls': freq_controls
        })
        
# Return
df_results = pd.DataFrame(results)
df_results['SNP'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])

#Print dimensions of the df_results dataframe
print(f'df_results shape: {df_results.shape}')
          
#Merge with the assoc file
sig_merge = freq[['SNP','A1','F_A','F_U','A2','L95','OR','U95','P']]
merged = pd.merge(df_results, sig_merge, on='SNP', how='right')
    
#Print dimensions of the merged dataframe (just adding more columns)
print(f'Merged dataframe shape: {merged.shape}') 
    
## Save to CSV
merged.to_csv(f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_assoc.txt', sep = '\t', index=False)

#### glm - Adjusting for age

In [ ]:
#Run case-control analysis for all variants with covariates (including age)

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

! /home/jupyter/plink2 \
--bfile {WORK_DIR}/PGLYRP2 \
--glm \
--adjust \
--maf 0.01 \
--mac 2 \
--ci 0.95 \
--hwe 0.0001 \
--covar {WORK_DIR}/AMPPD_EUR.COVS.txt \
--covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5 \
--covar-variance-standardize \
--out {WORK_DIR}/PGLYRP2.allvariants_age
    
#--recode A creates a new text fileset, showing each variant in each case and control for the minor allele (A). 
! /home/jupyter/plink1.9 \
--bfile {WORK_DIR}/PGLYRP2 \
--recode A \
--out {WORK_DIR}/PGLYRP2.allvariants_age

In [ ]:
#Process results from plink glm analysis for ALL variants and all covariates
#As there are very few or no significant variants with p-value < 0.05 - we will save results dataframe of all coding variants

print(f'WORKING ON: AMP-PD')
    
#Read in glm results
assoc = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_age.PHENO1.glm.logistic.hybrid', delim_whitespace=True)
assoc_add = assoc[assoc['TEST']=="ADD"]
    
#Filter for significant variants p < 0.05 - if any
significant = assoc_add[assoc_add['P']<0.05]
print(f'There are {len(significant)} variants with p-value < 0.05 in glm')

#Read in plink recoded data (.raw file)
recode = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_age.raw', delim_whitespace=True)

# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in AMP-PD for PGLYRP2: {len(variants)}')

# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]

results = []

# Pre-filter the dataset
total_cases = cases_data.shape[0]
total_controls = controls_data.shape[0]
results = []

for variant in variants:
    ## For PD cases
    hom_cases = (cases_data[variant] == 2).sum()
    het_cases = (cases_data[variant] == 1).sum()
    hom_ref_cases = (cases_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_cases = total_cases - (hom_cases + het_cases + hom_ref_cases)  # Missing data count
    freq_cases = (2 * hom_cases + het_cases) / (2 * (total_cases - missing_cases))  # Adjust for missing data in denominator

    ## For controls
    hom_controls = (controls_data[variant] == 2).sum()
    het_controls = (controls_data[variant] == 1).sum()
    hom_ref_controls = (controls_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_controls = total_controls - (hom_controls + het_controls + hom_ref_controls)  # Missing data count
    freq_controls = (2 * hom_controls + het_controls) / (2 * (total_controls - missing_controls))  # Adjust for missing data in denominator
    
    # Append results in dictionary format
    results.append({
        'Variant': variant,
        'Hom Cases': hom_cases,
        'Het Cases': het_cases,
        'Hom Ref Cases': hom_ref_cases,
        'Missing Cases': missing_cases,
        'Total Cases': total_cases,
        'Carrier Freq in Cases': freq_cases,
        'Hom Controls': hom_controls,
        'Het Controls': het_controls,
        'Hom Ref Controls': hom_ref_controls,
        'Missing Controls': missing_controls,
        'Total Controls': total_controls,
        'Carrier Freq in Controls': freq_controls
     })

# Return
df_results = pd.DataFrame(results)
df_results['ID'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])

#Print dimensions of the df_results dataframe
print(f'df_results shape: {df_results.shape}')
    
#Merge with the glm file
sig_merge = assoc_add[['ID','A1','A1_FREQ','OBS_CT','L95','OR','U95','LOG(OR)_SE','Z_STAT','P']]
merged = pd.merge(df_results, sig_merge, on='ID', how='right')
    
#Print dimensions of the merged dataframe (just adding more columns)
print(f'Merged dataframe shape: {merged.shape}')

## Save to CSV
merged.to_csv(f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_age.txt', sep = '\t', index=False)

In [ ]:
##Look at the glm file

glm_age = pd.read_csv('/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.allvariants_age.PHENO1.glm.logistic.hybrid',sep='\t')
glm_age.head()

Look at if there are any significant variants in the adjusted analysis:

In [ ]:
file_path = f'/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.allvariants_age.PHENO1.glm.logistic.hybrid.adjusted'

try:
    # Read file using whitespace as delimiter
    glm_age_adjust = pd.read_csv(file_path, delim_whitespace=True)

    # Sort the DataFrame by Bonferroni-corrected p-value (BONF), smallest to largest
    sorted_glm = glm_age_adjust.sort_values(by='BONF', ascending=True)

    print(f"\nTop entries for AMP-PD")
    print(sorted_glm.head())  # Use .to_string(index=False) for cleaner output if desired

except FileNotFoundError:
    print(f"File not found")

except Exception as e:
    print(f"Error processing AMP-PD: {e}")

#### glm - Not adjusting for age

In [ ]:
#Run case-control analysis for all variants with covariates but without age

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

! /home/jupyter/plink2 \
--bfile {WORK_DIR}/PGLYRP2 \
--glm \
--adjust \
--maf 0.01 \
--mac 2 \
--ci 0.95 \
--hwe 0.0001 \
--covar {WORK_DIR}/AMPPD_EUR.COVS.txt \
--covar-name SEX,PC1,PC2,PC3,PC4,PC5 \
--covar-variance-standardize \
--out {WORK_DIR}/PGLYRP2.allvariants_noage
    
#--recode A creates a new text fileset, showing each variant in each case and control for the minor allele (A). 
! /home/jupyter/plink1.9 \
--bfile {WORK_DIR}/PGLYRP2 \
--recode A \
--out {WORK_DIR}/PGLYRP2.allvariants_noage

In [ ]:
#Process results from plink glm analysis for ALL variants (without age as a covariate)
#As there are very few or no significant variants with p-value < 0.05 - we will save results dataframe of all coding variants
    
print(f'WORKING ON: AMP-PD')
    
#Read in glm results
assoc = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_noage.PHENO1.glm.logistic.hybrid', delim_whitespace=True)
assoc_add = assoc[assoc['TEST']=="ADD"]
    
#Filter for significant variants p < 0.05 - if any
significant = assoc_add[assoc_add['P']<0.05]
print(f'There are {len(significant)} variants with p-value < 0.05 in glm')
    
#Read in plink recoded data (.raw file)
recode = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_noage.raw', delim_whitespace=True)

# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in AMP-PD for PGLYRP2: {len(variants)}')

# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]

results = []

# Pre-filter the dataset
total_cases = cases_data.shape[0]
total_controls = controls_data.shape[0]
results = []

for variant in variants:
    ## For PD cases
    hom_cases = (cases_data[variant] == 2).sum()
    het_cases = (cases_data[variant] == 1).sum()
    hom_ref_cases = (cases_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_cases = total_cases - (hom_cases + het_cases + hom_ref_cases)  # Missing data count
    freq_cases = (2 * hom_cases + het_cases) / (2 * (total_cases - missing_cases))  # Adjust for missing data in denominator

    ## For controls
    hom_controls = (controls_data[variant] == 2).sum()
    het_controls = (controls_data[variant] == 1).sum()
    hom_ref_controls = (controls_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_controls = total_controls - (hom_controls + het_controls + hom_ref_controls)  # Missing data count
    freq_controls = (2 * hom_controls + het_controls) / (2 * (total_controls - missing_controls))  # Adjust for missing data in denominator
    
    # Append results in dictionary format
    results.append({
        'Variant': variant,
        'Hom Cases': hom_cases,
        'Het Cases': het_cases,
        'Hom Ref Cases': hom_ref_cases,
        'Missing Cases': missing_cases,
        'Total Cases': total_cases,
        'Carrier Freq in Cases': freq_cases,
        'Hom Controls': hom_controls,
        'Het Controls': het_controls,
        'Hom Ref Controls': hom_ref_controls,
        'Missing Controls': missing_controls,
        'Total Controls': total_controls,
        'Carrier Freq in Controls': freq_controls
    })

# Return
df_results = pd.DataFrame(results)
df_results['ID'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])

#Print dimensions of the df_results dataframe
print(f'df_results shape: {df_results.shape}')
    
#Merge with the glm file
sig_merge = assoc_add[['ID','A1','A1_FREQ','OBS_CT','L95','OR','U95','LOG(OR)_SE','Z_STAT','P']]
merged = pd.merge(df_results, sig_merge, on='ID', how='right')
    
#Print dimensions of the merged dataframe (just adding more columns)
print(f'Merged dataframe shape: {merged.shape}')
    
## Save to CSV
merged.to_csv(f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_noage.txt', sep = '\t', index=False)


In [ ]:
##Look at the glm file

glm_noage = pd.read_csv('/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.allvariants_noage.PHENO1.glm.logistic.hybrid',sep='\t')
glm_noage.head()

Look at if there are any significant variants in the adjusted analysis:

In [ ]:
file_path = f'/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.allvariants_noage.PHENO1.glm.logistic.hybrid.adjusted'

try:
    # Read file using whitespace as delimiter
    glm_age_adjust = pd.read_csv(file_path, delim_whitespace=True)

    # Sort the DataFrame by Bonferroni-corrected p-value (BONF), smallest to largest
    sorted_glm = glm_age_adjust.sort_values(by='BONF', ascending=True)

    print(f"\nTop entries for AMP-PD")
    print(sorted_glm.head())  # Use .to_string(index=False) for cleaner output if desired

except FileNotFoundError:
    print(f"File not found")

except Exception as e:
    print(f"Error processing AMP-PD: {e}")

## Extracting information on rs892145 for all ancestries + HWE

##### assoc

In [ ]:
# Extract information on the variant of interest (rs892145) from the association results

# Variant to search for
target_variant = 'chr19:15475861:A:T'

filename = f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_assoc.txt'

try:
    df = pd.read_csv(filename, sep='\t')

    # Filter the row where Variant column contains the target variant
    match = df[df['Variant'].str.contains(target_variant, na=False)]

    if not match.empty:
        print("AMPPD:")
        print(match.to_string(index=False))
        print("\n" + "="*80 + "\n")
    else:
        print("No match found in AMPPD.")
except FileNotFoundError:
    print(f"File not found: {filename}")
except Exception as e:
    print(f"Error processing {filename}: {e}")

##### glm - age

In [ ]:
#Print the results from the glm analyses for the variant of interest

# Variant to search for

target_variant = 'chr19:15475861:A:T'

filename = f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_age.txt'

try:
    df = pd.read_csv(filename, sep='\t')

    # Filter the row where Variant column contains the target variant
    match = df[df['Variant'].str.contains(target_variant, na=False)]

    if not match.empty:
        print("AMPPD:")
        print(match.to_string(index=False))
        print("\n" + "="*80 + "\n")
    else:
        print("No match found in AMPPD.")
except FileNotFoundError:
    print(f"File not found: {filename}")
except Exception as e:
    print(f"Error processing {filename}: {e}")

In [ ]:
#Print the adjusted p-value information from the glm analyses for the variant of interest

# Target variant and file path for AMPPD
target_variant = 'chr19:15475861:A:T'
filename = '/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.allvariants_age.PHENO1.glm.logistic.hybrid.adjusted'

try:
    df = pd.read_csv(filename, sep='\t')

    # Filter the row where the ID column contains the target variant
    match = df[df['ID'].str.contains(target_variant, na=False)]

    if not match.empty:
        print("AMPPD:")
        print(match.to_string(index=False))
        print("\n" + "="*80 + "\n")
    else:
        print("No match found in AMPPD.")
except FileNotFoundError:
    print(f"File not found: {filename}")
except Exception as e:
    print(f"Error processing {filename}: {e}")

##### glm - no age

In [ ]:
#Print the results from the glm analyses for the variant of interest

# Variant to search for

target_variant = 'chr19:15475861:A:T'

filename = f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_noage.txt'

try:
    df = pd.read_csv(filename, sep='\t')

    # Filter the row where Variant column contains the target variant
    match = df[df['Variant'].str.contains(target_variant, na=False)]

    if not match.empty:
        print("AMPPD:")
        print(match.to_string(index=False))
        print("\n" + "="*80 + "\n")
    else:
        print("No match found in AMPPD.")
except FileNotFoundError:
    print(f"File not found: {filename}")
except Exception as e:
    print(f"Error processing {filename}: {e}")

In [ ]:
#Print the adjusted p-value information from the glm analyses for the variant of interest

target_variant = 'chr19:15475861:A:T'
filename = '/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.allvariants_noage.PHENO1.glm.logistic.hybrid.adjusted'

try:
    df = pd.read_csv(filename, sep='\t')

    # Filter the row where the ID column contains the target variant
    match = df[df['ID'].str.contains(target_variant, na=False)]

    if not match.empty:
        print("AMPPD:")
        print(match.to_string(index=False))
        print("\n" + "="*80 + "\n")
    else:
        print("No match found in AMPPD.")
except FileNotFoundError:
    print(f"File not found: {filename}")
except Exception as e:
    print(f"Error processing {filename}: {e}")

#### HWE - rs892145

In [ ]:
## extract the SNP of interst and calculate HWE (controls only)

! /home/jupyter/plink2 \
--bfile {WORK_DIR}/PGLYRP2 \
--chr 19 \
--from-bp 15475861 \
--to-bp 15475861 \
--keep-if PHENO1==1 \
--hardy \
--out {WORK_DIR}/PGLYRP2_HWE

In [ ]:
##Look at the HWE file

hwe = pd.read_csv('/home/jupyter/PGLYRP2_AMPPD/PGLYRP2_HWE.hardy',sep='\t')
hwe.head()

## Sex stratified analyses
#### assoc - males

In [ ]:
#Run case-control analysis using plink assoc for ALL variants, not adjusting for any covariates and only in males (=1)

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

!/home/jupyter/plink1.9 \
--bfile {WORK_DIR}/PGLYRP2 \
--assoc \
--maf 0.01 \
--filter-males \
--mac 2 \
--hwe 0.0001 \
--adjust \
--allow-no-sex \
--ci 0.95 \
--out {WORK_DIR}/PGLYRP2.allvariants_males

#--recode A creates a new text fileset, showing each variant in each case and control for the minor allele (A).
! /home/jupyter/plink1.9 \
--bfile {WORK_DIR}/PGLYRP2 \
--filter-males \
--recode A \
--out {WORK_DIR}/PGLYRP2.allvariants_males


In [ ]:
#Process results from plink assoc unadjusted analysis
#As there are very few or no significant variants with p-value < 0.05 - we will save results dataframe of all coding variants

#Look at assoc results
freq = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_males.assoc', delim_whitespace=True)
    
#Filter for significant variants p < 0.05 - if any
sig_all_nonadj = freq[freq['P']<0.05]
    
print(f'There are {len(sig_all_nonadj)} variants with p-value < 0.05')

#Read in plink recoded data (.raw file)
recode = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_males.raw', delim_whitespace=True)

# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in AMP-PD for PGLYRP2: {len(variants)}')

# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]

results = []

# Pre-filter the dataset
total_cases = cases_data.shape[0]
total_controls = controls_data.shape[0]
results = []

for variant in variants:
    ## For PD cases
    hom_cases = (cases_data[variant] == 2).sum()
    het_cases = (cases_data[variant] == 1).sum()
    hom_ref_cases = (cases_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_cases = total_cases - (hom_cases + het_cases + hom_ref_cases)  # Missing data count
    freq_cases = (2 * hom_cases + het_cases) / (2 * (total_cases - missing_cases))  # Adjust for missing data in denominator

    ## For controls
    hom_controls = (controls_data[variant] == 2).sum()
    het_controls = (controls_data[variant] == 1).sum()
    hom_ref_controls = (controls_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_controls = total_controls - (hom_controls + het_controls + hom_ref_controls)  # Missing data count
    freq_controls = (2 * hom_controls + het_controls) / (2 * (total_controls - missing_controls))  # Adjust for missing data in denominator
    
    # Append results in dictionary format
    results.append({
        'Variant': variant,
        'Hom Cases': hom_cases,
        'Het Cases': het_cases,
        'Hom Ref Cases': hom_ref_cases,
        'Missing Cases': missing_cases,
        'Total Cases': total_cases,
        'Carrier Freq in Cases': freq_cases,
        'Hom Controls': hom_controls,
        'Het Controls': het_controls,
        'Hom Ref Controls': hom_ref_controls,
        'Missing Controls': missing_controls,
        'Total Controls': total_controls,
        'Carrier Freq in Controls': freq_controls
        })
        
# Return
df_results = pd.DataFrame(results)
df_results['SNP'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])

#Print dimensions of the df_results dataframe
print(f'df_results shape: {df_results.shape}')
          
#Merge with the assoc file
sig_merge = freq[['SNP','A1','F_A','F_U','A2','L95','OR','U95','P']]
merged = pd.merge(df_results, sig_merge, on='SNP', how='right')
    
#Print dimensions of the merged dataframe (just adding more columns)
print(f'Merged dataframe shape: {merged.shape}') 
    
## Save to CSV
merged.to_csv(f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_assoc_males.txt', sep = '\t', index=False)

#### glm - males - age

In [ ]:
#Run case-control analysis for all variants with covariates (including age) only in males

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

! /home/jupyter/plink2 \
--bfile {WORK_DIR}/PGLYRP2 \
--glm \
--adjust \
--maf 0.01 \
--filter-males \
--mac 2 \
--ci 0.95 \
--hwe 0.0001 \
--covar {WORK_DIR}/AMPPD_EUR.COVS.txt \
--covar-name AGE,PC1,PC2,PC3,PC4,PC5 \
--covar-variance-standardize \
--out {WORK_DIR}/PGLYRP2.allvariants_age_males
    
#--recode A creates a new text fileset, showing each variant in each case and control for the minor allele (A). 
! /home/jupyter/plink1.9 \
--bfile {WORK_DIR}/PGLYRP2 \
--filter-males \
--recode A \
--out {WORK_DIR}/PGLYRP2.allvariants_age_males

In [ ]:
#Process results from plink glm analysis for ALL variants and all covariates run only in males
#As there are very few or no significant variants with p-value < 0.05 - we will save results dataframe of all coding variants

print(f'WORKING ON: AMP-PD')
    
#Read in glm results
assoc = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_age_males.PHENO1.glm.logistic.hybrid', delim_whitespace=True)
assoc_add = assoc[assoc['TEST']=="ADD"]
    
#Filter for significant variants p < 0.05 - if any
significant = assoc_add[assoc_add['P']<0.05]
print(f'There are {len(significant)} variants with p-value < 0.05 in glm')

#Read in plink recoded data (.raw file)
recode = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_age_males.raw', delim_whitespace=True)

# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in AMP-PD for PGLYRP2: {len(variants)}')

# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]

results = []

# Pre-filter the dataset
total_cases = cases_data.shape[0]
total_controls = controls_data.shape[0]
results = []

for variant in variants:
    ## For PD cases
    hom_cases = (cases_data[variant] == 2).sum()
    het_cases = (cases_data[variant] == 1).sum()
    hom_ref_cases = (cases_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_cases = total_cases - (hom_cases + het_cases + hom_ref_cases)  # Missing data count
    freq_cases = (2 * hom_cases + het_cases) / (2 * (total_cases - missing_cases))  # Adjust for missing data in denominator

    ## For controls
    hom_controls = (controls_data[variant] == 2).sum()
    het_controls = (controls_data[variant] == 1).sum()
    hom_ref_controls = (controls_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_controls = total_controls - (hom_controls + het_controls + hom_ref_controls)  # Missing data count
    freq_controls = (2 * hom_controls + het_controls) / (2 * (total_controls - missing_controls))  # Adjust for missing data in denominator
    
    # Append results in dictionary format
    results.append({
        'Variant': variant,
        'Hom Cases': hom_cases,
        'Het Cases': het_cases,
        'Hom Ref Cases': hom_ref_cases,
        'Missing Cases': missing_cases,
        'Total Cases': total_cases,
        'Carrier Freq in Cases': freq_cases,
        'Hom Controls': hom_controls,
        'Het Controls': het_controls,
        'Hom Ref Controls': hom_ref_controls,
        'Missing Controls': missing_controls,
        'Total Controls': total_controls,
        'Carrier Freq in Controls': freq_controls
     })

# Return
df_results = pd.DataFrame(results)
df_results['ID'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])

#Print dimensions of the df_results dataframe
print(f'df_results shape: {df_results.shape}')
    
#Merge with the glm file
sig_merge = assoc_add[['ID','A1','A1_FREQ','OBS_CT','L95','OR','U95','LOG(OR)_SE','Z_STAT','P']]
merged = pd.merge(df_results, sig_merge, on='ID', how='right')
    
#Print dimensions of the merged dataframe (just adding more columns)
print(f'Merged dataframe shape: {merged.shape}')

## Save to CSV
merged.to_csv(f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_age_males.txt', sep = '\t', index=False)

Look at if there are any significant variants in the adjusted analysis:

In [ ]:
file_path = f'/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.allvariants_age_males.PHENO1.glm.logistic.hybrid.adjusted'

try:
    # Read file using whitespace as delimiter
    glm_age_adjust = pd.read_csv(file_path, delim_whitespace=True)

    # Sort the DataFrame by Bonferroni-corrected p-value (BONF), smallest to largest
    sorted_glm = glm_age_adjust.sort_values(by='BONF', ascending=True)

    print(f"\nTop entries for AMP-PD")
    print(sorted_glm.head())  # Use .to_string(index=False) for cleaner output if desired

except FileNotFoundError:
    print(f"File not found")

except Exception as e:
    print(f"Error processing AMP-PD: {e}")

#### glm - males - no age

In [ ]:
#Run case-control analysis for all variants with covariates but without age only in males

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

! /home/jupyter/plink2 \
--bfile {WORK_DIR}/PGLYRP2 \
--glm \
--adjust \
--maf 0.01 \
--filter-males \
--mac 2 \
--ci 0.95 \
--hwe 0.0001 \
--covar {WORK_DIR}/AMPPD_EUR.COVS.txt \
--covar-name PC1,PC2,PC3,PC4,PC5 \
--covar-variance-standardize \
--out {WORK_DIR}/PGLYRP2.allvariants_noage_males
    
#--recode A creates a new text fileset, showing each variant in each case and control for the minor allele (A). 
! /home/jupyter/plink1.9 \
--bfile {WORK_DIR}/PGLYRP2 \
--recode A \
--filter-males \
--out {WORK_DIR}/PGLYRP2.allvariants_noage_males

In [ ]:
#Process results from plink glm analysis for ALL variants (without age as a covariate)
#As there are very few or no significant variants with p-value < 0.05 - we will save results dataframe of all coding variants
    
print(f'WORKING ON: AMP-PD')
    
#Read in glm results
assoc = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_noage_males.PHENO1.glm.logistic.hybrid', delim_whitespace=True)
assoc_add = assoc[assoc['TEST']=="ADD"]
    
#Filter for significant variants p < 0.05 - if any
significant = assoc_add[assoc_add['P']<0.05]
print(f'There are {len(significant)} variants with p-value < 0.05 in glm')
    
#Read in plink recoded data (.raw file)
recode = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_noage_males.raw', delim_whitespace=True)

# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in AMP-PD for PGLYRP2: {len(variants)}')

# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]

results = []

# Pre-filter the dataset
total_cases = cases_data.shape[0]
total_controls = controls_data.shape[0]
results = []

for variant in variants:
    ## For PD cases
    hom_cases = (cases_data[variant] == 2).sum()
    het_cases = (cases_data[variant] == 1).sum()
    hom_ref_cases = (cases_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_cases = total_cases - (hom_cases + het_cases + hom_ref_cases)  # Missing data count
    freq_cases = (2 * hom_cases + het_cases) / (2 * (total_cases - missing_cases))  # Adjust for missing data in denominator

    ## For controls
    hom_controls = (controls_data[variant] == 2).sum()
    het_controls = (controls_data[variant] == 1).sum()
    hom_ref_controls = (controls_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_controls = total_controls - (hom_controls + het_controls + hom_ref_controls)  # Missing data count
    freq_controls = (2 * hom_controls + het_controls) / (2 * (total_controls - missing_controls))  # Adjust for missing data in denominator
    
    # Append results in dictionary format
    results.append({
        'Variant': variant,
        'Hom Cases': hom_cases,
        'Het Cases': het_cases,
        'Hom Ref Cases': hom_ref_cases,
        'Missing Cases': missing_cases,
        'Total Cases': total_cases,
        'Carrier Freq in Cases': freq_cases,
        'Hom Controls': hom_controls,
        'Het Controls': het_controls,
        'Hom Ref Controls': hom_ref_controls,
        'Missing Controls': missing_controls,
        'Total Controls': total_controls,
        'Carrier Freq in Controls': freq_controls
    })

# Return
df_results = pd.DataFrame(results)
df_results['ID'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])

#Print dimensions of the df_results dataframe
print(f'df_results shape: {df_results.shape}')
    
#Merge with the glm file
sig_merge = assoc_add[['ID','A1','A1_FREQ','OBS_CT','L95','OR','U95','LOG(OR)_SE','Z_STAT','P']]
merged = pd.merge(df_results, sig_merge, on='ID', how='right')
    
#Print dimensions of the merged dataframe (just adding more columns)
print(f'Merged dataframe shape: {merged.shape}')
    
## Save to CSV
merged.to_csv(f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_noage_males.txt', sep = '\t', index=False)


Look at if there are any significant variants in the adjusted analysis:

In [ ]:
file_path = f'/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.allvariants_noage_males.PHENO1.glm.logistic.hybrid.adjusted'

try:
    # Read file using whitespace as delimiter
    glm_age_adjust = pd.read_csv(file_path, delim_whitespace=True)

    # Sort the DataFrame by Bonferroni-corrected p-value (BONF), smallest to largest
    sorted_glm = glm_age_adjust.sort_values(by='BONF', ascending=True)

    print(f"\nTop entries for AMP-PD")
    print(sorted_glm.head())  # Use .to_string(index=False) for cleaner output if desired

except FileNotFoundError:
    print(f"File not found")

except Exception as e:
    print(f"Error processing AMP-PD: {e}")

#### assoc - females

In [ ]:
#Run case-control analysis using plink assoc for ALL variants, not adjusting for any covariates and only in females (=2)

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

!/home/jupyter/plink1.9 \
--bfile {WORK_DIR}/PGLYRP2 \
--assoc \
--maf 0.01 \
--mac 2 \
--hwe 0.0001 \
--filter-females \
--adjust \
--allow-no-sex \
--ci 0.95 \
--out {WORK_DIR}/PGLYRP2.allvariants_females
    
#--recode A creates a new text fileset, showing each variant in each case and control for the minor allele (A).
! /home/jupyter/plink1.9 \
--bfile {WORK_DIR}/PGLYRP2 \
--recode A \
--filter-females \
--out {WORK_DIR}/PGLYRP2.allvariants_females

In [ ]:
#Process results from plink assoc unadjusted analysis in females
#As there are very few or no significant variants with p-value < 0.05 - we will save results dataframe of all coding variants

#Look at assoc results
freq = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_females.assoc', delim_whitespace=True)
    
#Filter for significant variants p < 0.05 - if any
sig_all_nonadj = freq[freq['P']<0.05]
    
print(f'There are {len(sig_all_nonadj)} variants with p-value < 0.05')

#Read in plink recoded data (.raw file)
recode = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_females.raw', delim_whitespace=True)

# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in AMP-PD for PGLYRP2: {len(variants)}')

# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]

results = []

# Pre-filter the dataset
total_cases = cases_data.shape[0]
total_controls = controls_data.shape[0]
results = []

for variant in variants:
    ## For PD cases
    hom_cases = (cases_data[variant] == 2).sum()
    het_cases = (cases_data[variant] == 1).sum()
    hom_ref_cases = (cases_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_cases = total_cases - (hom_cases + het_cases + hom_ref_cases)  # Missing data count
    freq_cases = (2 * hom_cases + het_cases) / (2 * (total_cases - missing_cases))  # Adjust for missing data in denominator

    ## For controls
    hom_controls = (controls_data[variant] == 2).sum()
    het_controls = (controls_data[variant] == 1).sum()
    hom_ref_controls = (controls_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_controls = total_controls - (hom_controls + het_controls + hom_ref_controls)  # Missing data count
    freq_controls = (2 * hom_controls + het_controls) / (2 * (total_controls - missing_controls))  # Adjust for missing data in denominator
    
    # Append results in dictionary format
    results.append({
        'Variant': variant,
        'Hom Cases': hom_cases,
        'Het Cases': het_cases,
        'Hom Ref Cases': hom_ref_cases,
        'Missing Cases': missing_cases,
        'Total Cases': total_cases,
        'Carrier Freq in Cases': freq_cases,
        'Hom Controls': hom_controls,
        'Het Controls': het_controls,
        'Hom Ref Controls': hom_ref_controls,
        'Missing Controls': missing_controls,
        'Total Controls': total_controls,
        'Carrier Freq in Controls': freq_controls
        })
        
# Return
df_results = pd.DataFrame(results)
df_results['SNP'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])

#Print dimensions of the df_results dataframe
print(f'df_results shape: {df_results.shape}')
          
#Merge with the assoc file
sig_merge = freq[['SNP','A1','F_A','F_U','A2','L95','OR','U95','P']]
merged = pd.merge(df_results, sig_merge, on='SNP', how='right')
    
#Print dimensions of the merged dataframe (just adding more columns)
print(f'Merged dataframe shape: {merged.shape}') 
    
## Save to CSV
merged.to_csv(f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_assoc_females.txt', sep = '\t', index=False)

#### glm - females - age

In [ ]:
#Run case-control analysis for all variants with covariates in females only

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

! /home/jupyter/plink2 \
--bfile {WORK_DIR}/PGLYRP2 \
--glm \
--adjust \
--maf 0.01 \
--filter-females \
--mac 2 \
--ci 0.95 \
--hwe 0.0001 \
--covar {WORK_DIR}/AMPPD_EUR.COVS.txt \
--covar-name AGE,PC1,PC2,PC3,PC4,PC5 \
--covar-variance-standardize \
--out {WORK_DIR}/PGLYRP2.allvariants_age_females
    
#--recode A creates a new text fileset, showing each variant in each case and control for the minor allele (A). 
! /home/jupyter/plink1.9 \
--bfile {WORK_DIR}/PGLYRP2 \
--recode A \
--filter-females \
--out {WORK_DIR}/PGLYRP2.allvariants_age_females

In [ ]:
#Process results from plink glm analysis for ALL variants and all covariates in females only
#As there are very few or no significant variants with p-value < 0.05 - we will save results dataframe of all coding variants

print(f'WORKING ON: AMP-PD')
    
#Read in glm results
assoc = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_age_females.PHENO1.glm.logistic.hybrid', delim_whitespace=True)
assoc_add = assoc[assoc['TEST']=="ADD"]
    
#Filter for significant variants p < 0.05 - if any
significant = assoc_add[assoc_add['P']<0.05]
print(f'There are {len(significant)} variants with p-value < 0.05 in glm')

#Read in plink recoded data (.raw file)
recode = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_age_females.raw', delim_whitespace=True)

# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in AMP-PD for PGLYRP2: {len(variants)}')

# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]

results = []

# Pre-filter the dataset
total_cases = cases_data.shape[0]
total_controls = controls_data.shape[0]
results = []

for variant in variants:
    ## For PD cases
    hom_cases = (cases_data[variant] == 2).sum()
    het_cases = (cases_data[variant] == 1).sum()
    hom_ref_cases = (cases_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_cases = total_cases - (hom_cases + het_cases + hom_ref_cases)  # Missing data count
    freq_cases = (2 * hom_cases + het_cases) / (2 * (total_cases - missing_cases))  # Adjust for missing data in denominator

    ## For controls
    hom_controls = (controls_data[variant] == 2).sum()
    het_controls = (controls_data[variant] == 1).sum()
    hom_ref_controls = (controls_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_controls = total_controls - (hom_controls + het_controls + hom_ref_controls)  # Missing data count
    freq_controls = (2 * hom_controls + het_controls) / (2 * (total_controls - missing_controls))  # Adjust for missing data in denominator
    
    # Append results in dictionary format
    results.append({
        'Variant': variant,
        'Hom Cases': hom_cases,
        'Het Cases': het_cases,
        'Hom Ref Cases': hom_ref_cases,
        'Missing Cases': missing_cases,
        'Total Cases': total_cases,
        'Carrier Freq in Cases': freq_cases,
        'Hom Controls': hom_controls,
        'Het Controls': het_controls,
        'Hom Ref Controls': hom_ref_controls,
        'Missing Controls': missing_controls,
        'Total Controls': total_controls,
        'Carrier Freq in Controls': freq_controls
     })

# Return
df_results = pd.DataFrame(results)
df_results['ID'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])

#Print dimensions of the df_results dataframe
print(f'df_results shape: {df_results.shape}')
    
#Merge with the glm file
sig_merge = assoc_add[['ID','A1','A1_FREQ','OBS_CT','L95','OR','U95','LOG(OR)_SE','Z_STAT','P']]
merged = pd.merge(df_results, sig_merge, on='ID', how='right')
    
#Print dimensions of the merged dataframe (just adding more columns)
print(f'Merged dataframe shape: {merged.shape}')

## Save to CSV
merged.to_csv(f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_age_females.txt', sep = '\t', index=False)

Look at if there are any significant variants in the adjusted analysis:

In [ ]:
file_path = f'/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.allvariants_age_females.PHENO1.glm.logistic.hybrid.adjusted'

try:
    # Read file using whitespace as delimiter
    glm_age_adjust = pd.read_csv(file_path, delim_whitespace=True)

    # Sort the DataFrame by Bonferroni-corrected p-value (BONF), smallest to largest
    sorted_glm = glm_age_adjust.sort_values(by='BONF', ascending=True)

    print(f"\nTop entries for AMP-PD")
    print(sorted_glm.head())  # Use .to_string(index=False) for cleaner output if desired

except FileNotFoundError:
    print(f"File not found")

except Exception as e:
    print(f"Error processing AMP-PD: {e}")

#### glm - females - no age

In [ ]:
#Run case-control analysis for all variants with covariates in females only

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

! /home/jupyter/plink2 \
--bfile {WORK_DIR}/PGLYRP2 \
--glm \
--adjust \
--maf 0.01 \
--filter-females \
--mac 2 \
--ci 0.95 \
--hwe 0.0001 \
--covar {WORK_DIR}/AMPPD_EUR.COVS.txt \
--covar-name PC1,PC2,PC3,PC4,PC5 \
--covar-variance-standardize \
--out {WORK_DIR}/PGLYRP2.allvariants_noage_females
    
#--recode A creates a new text fileset, showing each variant in each case and control for the minor allele (A). 
! /home/jupyter/plink1.9 \
--bfile {WORK_DIR}/PGLYRP2 \
--filter-females \
--recode A \
--out {WORK_DIR}/PGLYRP2.allvariants_noage_females

In [ ]:
#Process results from plink glm analysis for ALL variants (without age as a covariate) in females only
#As there are very few or no significant variants with p-value < 0.05 - we will save results dataframe of all coding variants
    
print(f'WORKING ON: AMP-PD')
    
#Read in glm results
assoc = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_noage_females.PHENO1.glm.logistic.hybrid', delim_whitespace=True)
assoc_add = assoc[assoc['TEST']=="ADD"]
    
#Filter for significant variants p < 0.05 - if any
significant = assoc_add[assoc_add['P']<0.05]
print(f'There are {len(significant)} variants with p-value < 0.05 in glm')
    
#Read in plink recoded data (.raw file)
recode = pd.read_csv(f'{WORK_DIR}/PGLYRP2.allvariants_noage_females.raw', delim_whitespace=True)

# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in AMP-PD for PGLYRP2: {len(variants)}')

# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]

results = []

# Pre-filter the dataset
total_cases = cases_data.shape[0]
total_controls = controls_data.shape[0]
results = []

for variant in variants:
    ## For PD cases
    hom_cases = (cases_data[variant] == 2).sum()
    het_cases = (cases_data[variant] == 1).sum()
    hom_ref_cases = (cases_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_cases = total_cases - (hom_cases + het_cases + hom_ref_cases)  # Missing data count
    freq_cases = (2 * hom_cases + het_cases) / (2 * (total_cases - missing_cases))  # Adjust for missing data in denominator

    ## For controls
    hom_controls = (controls_data[variant] == 2).sum()
    het_controls = (controls_data[variant] == 1).sum()
    hom_ref_controls = (controls_data[variant] == 0).sum()  # Homozygous reference genotype
    missing_controls = total_controls - (hom_controls + het_controls + hom_ref_controls)  # Missing data count
    freq_controls = (2 * hom_controls + het_controls) / (2 * (total_controls - missing_controls))  # Adjust for missing data in denominator
    
    # Append results in dictionary format
    results.append({
        'Variant': variant,
        'Hom Cases': hom_cases,
        'Het Cases': het_cases,
        'Hom Ref Cases': hom_ref_cases,
        'Missing Cases': missing_cases,
        'Total Cases': total_cases,
        'Carrier Freq in Cases': freq_cases,
        'Hom Controls': hom_controls,
        'Het Controls': het_controls,
        'Hom Ref Controls': hom_ref_controls,
        'Missing Controls': missing_controls,
        'Total Controls': total_controls,
        'Carrier Freq in Controls': freq_controls
    })

# Return
df_results = pd.DataFrame(results)
df_results['ID'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])

#Print dimensions of the df_results dataframe
print(f'df_results shape: {df_results.shape}')
    
#Merge with the glm file
sig_merge = assoc_add[['ID','A1','A1_FREQ','OBS_CT','L95','OR','U95','LOG(OR)_SE','Z_STAT','P']]
merged = pd.merge(df_results, sig_merge, on='ID', how='right')
    
#Print dimensions of the merged dataframe (just adding more columns)
print(f'Merged dataframe shape: {merged.shape}')
    
## Save to CSV
merged.to_csv(f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_noage_females.txt', sep = '\t', index=False)


Look at if there are any significant variants in the adjusted analysis:

In [ ]:
file_path = f'/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.allvariants_noage_females.PHENO1.glm.logistic.hybrid.adjusted'

try:
    # Read file using whitespace as delimiter
    glm_age_adjust = pd.read_csv(file_path, delim_whitespace=True)

    # Sort the DataFrame by Bonferroni-corrected p-value (BONF), smallest to largest
    sorted_glm = glm_age_adjust.sort_values(by='BONF', ascending=True)

    print(f"\nTop entries for AMP-PD")
    print(sorted_glm.head())  # Use .to_string(index=False) for cleaner output if desired

except FileNotFoundError:
    print(f"File not found")

except Exception as e:
    print(f"Error processing AMP-PD: {e}")

### Extracting information on rs892145 only in males vs females

#### males

##### assoc

In [ ]:
# Extract information on the variant of interest (rs892145) from the association results in males

# Variant to search for
target_variant = 'chr19:15475861:A:T'

filename = f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_assoc_males.txt'

try:
    df = pd.read_csv(filename, sep='\t')

    # Filter the row where Variant column contains the target variant
    match = df[df['Variant'].str.contains(target_variant, na=False)]

    if not match.empty:
        print("AMPPD:")
        print(match.to_string(index=False))
        print("\n" + "="*80 + "\n")
    else:
        print("No match found in AMPPD.")
except FileNotFoundError:
    print(f"File not found: {filename}")
except Exception as e:
    print(f"Error processing {filename}: {e}")

##### glm - age

In [ ]:
#Print the results from the glm analyses for the variant of interest

# Variant to search for

target_variant = 'chr19:15475861:A:T'

filename = f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_age_males.txt'

try:
    df = pd.read_csv(filename, sep='\t')

    # Filter the row where Variant column contains the target variant
    match = df[df['Variant'].str.contains(target_variant, na=False)]

    if not match.empty:
        print("AMPPD:")
        print(match.to_string(index=False))
        print("\n" + "="*80 + "\n")
    else:
        print("No match found in AMPPD.")
except FileNotFoundError:
    print(f"File not found: {filename}")
except Exception as e:
    print(f"Error processing {filename}: {e}")

In [ ]:
#Print the adjusted p-value information from the glm analyses for the variant of interest in males only

# Target variant and file path for AMPPD
target_variant = 'chr19:15475861:A:T'
filename = '/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.allvariants_age_males.PHENO1.glm.logistic.hybrid.adjusted'

try:
    df = pd.read_csv(filename, sep='\t')

    # Filter the row where the ID column contains the target variant
    match = df[df['ID'].str.contains(target_variant, na=False)]

    if not match.empty:
        print("AMPPD:")
        print(match.to_string(index=False))
        print("\n" + "="*80 + "\n")
    else:
        print("No match found in AMPPD.")
except FileNotFoundError:
    print(f"File not found: {filename}")
except Exception as e:
    print(f"Error processing {filename}: {e}")

##### glm - no age

In [ ]:
#Print the results from the glm analyses for the variant of interest (without age as covariate) only in males

# Variant to search for

target_variant = 'chr19:15475861:A:T'

filename = f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_noage_males.txt'

try:
    df = pd.read_csv(filename, sep='\t')

    # Filter the row where Variant column contains the target variant
    match = df[df['Variant'].str.contains(target_variant, na=False)]

    if not match.empty:
        print("AMPPD:")
        print(match.to_string(index=False))
        print("\n" + "="*80 + "\n")
    else:
        print("No match found in AMPPD.")
except FileNotFoundError:
    print(f"File not found: {filename}")
except Exception as e:
    print(f"Error processing {filename}: {e}")

In [ ]:
#Print the adjusted p-value information from the glm analyses for the variant of interest

target_variant = 'chr19:15475861:A:T'
filename = '/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.allvariants_noage_males.PHENO1.glm.logistic.hybrid.adjusted'

try:
    df = pd.read_csv(filename, sep='\t')

    # Filter the row where the ID column contains the target variant
    match = df[df['ID'].str.contains(target_variant, na=False)]

    if not match.empty:
        print("AMPPD:")
        print(match.to_string(index=False))
        print("\n" + "="*80 + "\n")
    else:
        print("No match found in AMPPD.")
except FileNotFoundError:
    print(f"File not found: {filename}")
except Exception as e:
    print(f"Error processing {filename}: {e}")

#### females

##### assoc

In [ ]:
# Extract information on the variant of interest (rs892145) from the association results in males

# Variant to search for
target_variant = 'chr19:15475861:A:T'

filename = f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_assoc_females.txt'

try:
    df = pd.read_csv(filename, sep='\t')

    # Filter the row where Variant column contains the target variant
    match = df[df['Variant'].str.contains(target_variant, na=False)]

    if not match.empty:
        print("AMPPD:")
        print(match.to_string(index=False))
        print("\n" + "="*80 + "\n")
    else:
        print("No match found in AMPPD.")
except FileNotFoundError:
    print(f"File not found: {filename}")
except Exception as e:
    print(f"Error processing {filename}: {e}")

##### glm - age

In [ ]:
#Print the results from the glm analyses for the variant of interest in females

# Variant to search for

target_variant = 'chr19:15475861:A:T'

filename = f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_age_females.txt'

try:
    df = pd.read_csv(filename, sep='\t')

    # Filter the row where Variant column contains the target variant
    match = df[df['Variant'].str.contains(target_variant, na=False)]

    if not match.empty:
        print("AMPPD:")
        print(match.to_string(index=False))
        print("\n" + "="*80 + "\n")
    else:
        print("No match found in AMPPD.")
except FileNotFoundError:
    print(f"File not found: {filename}")
except Exception as e:
    print(f"Error processing {filename}: {e}")

In [ ]:
#Print the adjusted p-value information from the glm analyses for the variant of interest in females only

# Target variant and file path for AMPPD
target_variant = 'chr19:15475861:A:T'
filename = '/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.allvariants_age_females.PHENO1.glm.logistic.hybrid.adjusted'

try:
    df = pd.read_csv(filename, sep='\t')

    # Filter the row where the ID column contains the target variant
    match = df[df['ID'].str.contains(target_variant, na=False)]

    if not match.empty:
        print("AMPPD:")
        print(match.to_string(index=False))
        print("\n" + "="*80 + "\n")
    else:
        print("No match found in AMPPD.")
except FileNotFoundError:
    print(f"File not found: {filename}")
except Exception as e:
    print(f"Error processing {filename}: {e}")

##### glm - no age

In [ ]:
#Print the results from the glm analyses for the variant of interest (without age as covariate) only in females

# Variant to search for

target_variant = 'chr19:15475861:A:T'

filename = f'{WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_noage_females.txt'

try:
    df = pd.read_csv(filename, sep='\t')

    # Filter the row where Variant column contains the target variant
    match = df[df['Variant'].str.contains(target_variant, na=False)]

    if not match.empty:
        print("AMPPD:")
        print(match.to_string(index=False))
        print("\n" + "="*80 + "\n")
    else:
        print("No match found in AMPPD.")
except FileNotFoundError:
    print(f"File not found: {filename}")
except Exception as e:
    print(f"Error processing {filename}: {e}")

In [ ]:
#Print the adjusted p-value information from the glm analyses for the variant of interest in females

target_variant = 'chr19:15475861:A:T'
filename = '/home/jupyter/PGLYRP2_AMPPD/PGLYRP2.allvariants_noage_females.PHENO1.glm.logistic.hybrid.adjusted'

try:
    df = pd.read_csv(filename, sep='\t')

    # Filter the row where the ID column contains the target variant
    match = df[df['ID'].str.contains(target_variant, na=False)]

    if not match.empty:
        print("AMPPD:")
        print(match.to_string(index=False))
        print("\n" + "="*80 + "\n")
    else:
        print("No match found in AMPPD.")
except FileNotFoundError:
    print(f"File not found: {filename}")
except Exception as e:
    print(f"Error processing {filename}: {e}")

## Interaction analyses

In [ ]:
# Install R 

!pip install rpy2

In [ ]:
# Load rpy2 and activate the R interface

import rpy2.robjects as robjects


In [ ]:
# Test if R is working correctly 

robjects.r('R.version.string')

In [ ]:
# To use R natively in a cell, load the R magic extension

%load_ext rpy2.ipython

In [ ]:
!echo "chr19:15475861:A:T" > /home/jupyter/PGLYRP2_AMPPD/single_snp.txt
!echo "chr19:15475861:A:T T" > /home/jupyter/PGLYRP2_AMPPD/reference_allele.txt

In [ ]:
# Create a new covariate file for R where genotypes are coded additively (0,1,2)

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

!/home/jupyter/plink2 \
--bfile {WORK_DIR}/PGLYRP2 \
--extract {WORK_DIR}/single_snp.txt \
--pheno {WORK_DIR}/AMPPD_EUR.COVS.txt \
--pheno-name PHENO \
--covar {WORK_DIR}/AMPPD_EUR.COVS.txt \
--covar-name SEX,PC1,PC2,PC3,PC4,PC5 \
--ref-allele {WORK_DIR}/reference_allele.txt \
--recode A \
--out {WORK_DIR}/PGLYRP2_interaction
    


In [ ]:
!head {WORK_DIR}/PGLYRP2_interaction.raw

In [ ]:
!head {WORK_DIR}/PGLYRP2_interaction.cov

In [ ]:
#Merge the ANC_PGLYRP2_interaction.raw with the ANC_PGLYRP2_interaction.cov on IID
#Drop SEX from one of the files to avoid duplicates

cov_path = f'/home/jupyter/PGLYRP2_AMPPD/PGLYRP2_interaction.cov'
raw_path = f'/home/jupyter/PGLYRP2_AMPPD/PGLYRP2_interaction.raw'

try:
    # Load both files
    cov_df = pd.read_csv(cov_path, delim_whitespace=True)
    raw_df = pd.read_csv(raw_path, delim_whitespace=True)

    # Rename '#IID' to 'IID' for merging
    cov_df = cov_df.rename(columns={'#IID': 'IID'})

    # Drop 'SEX' if present to avoid duplication
    cov_df = cov_df.drop(columns=['SEX'], errors='ignore')

    # Merge on 'IID'
    merged_df = pd.merge(raw_df, cov_df, on='IID', how='inner')

    # Save merged DataFrame back to raw path
    merged_df.to_csv(raw_path, sep=' ', index=False)

    print(f"Saved merged file for AMPPD: {raw_path}")

except FileNotFoundError:
    print("Missing file for AMPPD")
except Exception as e:
    print(f"Error processing AMPPD: {e}")

In [ ]:
%%R

#prepare the file - transform pheno coding, put sex as categorical and set female as the reference group

file_path <- "/home/jupyter/PGLYRP2_AMPPD/PGLYRP2_interaction.raw"

# Check if file exists before processing
if (file.exists(file_path)) {
  # Read the file
  interaction_data <- read.table(file_path, header = TRUE)
  
  # Transform PHENOTYPE coding
  interaction_data$PHENOTYPE <- ifelse(interaction_data$PHENOTYPE == 2, 1, 0)
  
  # Transform SEX to categorical and relevel with "Female" as reference
  interaction_data$SEX <- factor(interaction_data$SEX, levels = c(1, 2), labels = c("Male", "Female"))
  interaction_data$SEX <- relevel(interaction_data$SEX, ref = "Female")
  
  # Print a quick summary or head
  cat("\nProcessed: AMPPD\n")
  print(head(interaction_data))
} else {
  cat("\nFile not found for AMPPD\n")
}

In [ ]:
%%R

# File path for AMPPD interaction file
file_path <- "/home/jupyter/PGLYRP2_AMPPD/PGLYRP2_interaction.raw"

if (file.exists(file_path)) {
  interaction_data <- read.table(file_path, header = TRUE)
  
  # Transform PHENOTYPE coding
  interaction_data$PHENOTYPE <- ifelse(interaction_data$PHENOTYPE == 2, 1, 0)
  
  # Transform SEX to categorical and relevel with "Female" as reference
  interaction_data$SEX <- factor(interaction_data$SEX, levels = c(1, 2), labels = c("Male", "Female"))
  interaction_data$SEX <- relevel(interaction_data$SEX, ref = "Female")
  
  # Fit logistic regression model with interaction term
  glm_interaction <- glm(PHENOTYPE ~ SEX * `chr19.15475861.A.T_T` + PC1+PC2+PC3+PC4+PC5, data = interaction_data, 
                         family = binomial)
  
  # Print model summary
  cat("\nAncestry: AMPPD\n")
  print(summary(glm_interaction))
  
  # Print coefficients with their names
  cat("\nCoefficients and names:\n")
  coeffs <- coef(glm_interaction)
  for (name in names(coeffs)) {
    cat(name, ": ", coeffs[name], "\n")
  }
  
  # Print odds ratios with 95% confidence intervals
  cat("\nOdds Ratios and 95% CI:\n")
  odds_ratios <- exp(cbind(coef(glm_interaction), suppressMessages(confint(glm_interaction))))
  print(odds_ratios)
  
  cat("\n", strrep("=", 80), "\n")
  
} else {
  cat("\nFile not found for AMPPD\n")
}



## Save files to the bucket

In [ ]:
## Save the final results to workspace bucket (move from VM to workspace bucket)

# Burden
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r {WORK_DIR}/PGLYRP2.burden.* {WORKSPACE_BUCKET}/PGLYRP2.burden/')



In [ ]:
## Save the final results to workspace bucket (move from VM to workspace bucket)

WORK_DIR = f'/home/jupyter/PGLYRP2_AMPPD'

# GLM results - adjusted
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r {WORK_DIR}/PGLYRP2.allvariants_age.PHENO1.glm.logistic.hybrid.adjusted {WORKSPACE_BUCKET}/PGLYRP2.allvariants_age.PHENO1.glm.logistic.hybrid.adjusted')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r {WORK_DIR}/PGLYRP2.allvariants_noage.PHENO1.glm.logistic.hybrid.adjusted {WORKSPACE_BUCKET}/PGLYRP2.allvariants_noage.PHENO1.glm.logistic.hybrid.adjusted')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r {WORK_DIR}/PGLYRP2.allvariants_age_males.PHENO1.glm.logistic.hybrid.adjusted {WORKSPACE_BUCKET}/PGLYRP2.allvariants_age_males.PHENO1.glm.logistic.hybrid.adjusted')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r {WORK_DIR}/PGLYRP2.allvariants_noage_males.PHENO1.glm.logistic.hybrid.adjusted {WORKSPACE_BUCKET}/PGLYRP2.allvariants_noage_males.PHENO1.glm.logistic.hybrid.adjusted')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r {WORK_DIR}/PGLYRP2.allvariants_age_females.PHENO1.glm.logistic.hybrid.adjusted {WORKSPACE_BUCKET}/PGLYRP2.allvariants_age_females.PHENO1.glm.logistic.hybrid.adjusted')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r {WORK_DIR}/PGLYRP2.allvariants_noage_females.PHENO1.glm.logistic.hybrid.adjusted {WORKSPACE_BUCKET}/PGLYRP2.allvariants_noage_females.PHENO1.glm.logistic.hybrid.adjusted')

# GLM results - glm
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r {WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_age.txt {WORKSPACE_BUCKET}/PGLYRP2.AMPPD.allvariants_glm_age.txt')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r {WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_noage.txt {WORKSPACE_BUCKET}/PGLYRP2.AMPPD.allvariants_glm_noage.txt')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r {WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_age_males.txt {WORKSPACE_BUCKET}/PGLYRP2.AMPPD.allvariants_glm_age_males.txt')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r {WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_age_females.txt {WORKSPACE_BUCKET}/PGLYRP2.AMPPD.allvariants_glm_age_females.txt')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r {WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_noage_males.txt {WORKSPACE_BUCKET}/PGLYRP2.AMPPD.allvariants_glm_noage_males.txt')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r {WORK_DIR}/PGLYRP2.AMPPD.allvariants_glm_noage_females.txt {WORKSPACE_BUCKET}/PGLYRP2.AMPPD.allvariants_glm_noage_females.txt')


